<a href="https://colab.research.google.com/github/gusjav35/iageneracionimg/blob/main/generador_photomaker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers transformers safetensors accelerate


In [ ]:
from PIL import Image

# Cargar imagen directamente desde el entorno de archivos
input_image = Image.open("selfie.png").convert("RGB")

In [ ]:
from diffusers import DiffusionPipeline
from google.colab import userdata
import torch

# Usar el token desde secretos
hf_token = userdata.get('HF_TOKEN')


In [ ]:
from pipeline import PhotoMakerStableDiffusionXLPipeline

In [ ]:
# Cargar modelo base compatible con PhotoMaker
pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
).to("cuda" if torch.cuda.is_available() else "cpu")

# Reemplazar clase base por clase PhotoMaker
pipe.__class__ = PhotoMakerStableDiffusionXLPipeline

# Cargar el adaptador de PhotoMaker
pipe.load_photomaker_adapter(
    pretrained_model_name_or_path_or_dict="TencentARC/PhotoMaker",
    weight_name="photomaker-v1.bin",
    trigger_word="img"
)

In [ ]:
# Variable modificable
especialidad = "radiology"  # Cambia esto por la especialidad deseada (e.g., "pediatrician", "cardiologist")

# Prompt mejorado para reflejar actividad profesional
prompt = (
    f"a full-body realistic photo of a young male {especialidad} actively performing tasks in a hospital setting, "
    f"such as examining a patient, writing on a chart, or using medical instruments, "
    "wearing appropriate medical uniform, DSLR quality, natural lighting, ultra realistic, professional, img"
)

# Negative prompt para evitar estilos no realistas
negative_prompt = (
    "cartoon, anime, illustration, drawing, painting, 3d, cgi, bad anatomy, extra limbs, disfigured face, "
    "blurry, low resolution, distorted eyes, asymmetrical face, skin blemishes, unrealistic skin texture, grainy, jpeg artifacts"
)



result = pipe(
    prompt=prompt,
    input_id_images=input_image,
    negative_prompt=negative_prompt,
    num_inference_steps=30,
    guidance_scale=5.0
)


# Guardar resultado
result.images[0].save("output.png")

In [ ]:
from IPython.display import Image as IPyImage, display

# Mostrar imagen generada
display(IPyImage(filename="output.png"))


In [ ]:
#Liberar memoria si ya no voy a generar imágenes
del pipe
gc.collect()
if torch.cuda.is_available():
    torch.cuda.empty_cache()
